In [ ]:
!pip install pennylane==0.33 pennylane-qiskit==0.33 qiskit qiskit-ibm-provider --quiet

In [ ]:
!pip install torch

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import os

In [ ]:
# 📌 Load Data
df = pd.read_csv("pose_features_master.csv")

# 🚫 Drop Non-Gait Features
gait_features = [
       'Left Shoulder_Mean',
       'Left Shoulder_Std', 'Right Shoulder_Mean', 'Right Shoulder_Std',
       'Left Elbow_Mean', 'Left Elbow_Std', 'Right Elbow_Mean',
       'Right Elbow_Std', 'Left Wrist_Mean', 'Left Wrist_Std',
       'Right Wrist_Mean', 'Right Wrist_Std', 'Left Hip_Mean', 'Left Hip_Std',
       'Right Hip_Mean', 'Right Hip_Std', 'Left Knee_Mean', 'Left Knee_Std',
       'Right Knee_Mean', 'Right Knee_Std', 'Left Ankle_Mean',
       'Left Ankle_Std', 'Right Ankle_Mean', 'Right Ankle_Std',
       'Left Shoulder_Velocity_Mean',
       'Left Shoulder_Acceleration_Mean', 'Left Shoulder_Jerk_Mean',
       'Right Shoulder_Velocity_Mean', 'Right Shoulder_Acceleration_Mean',
       'Right Shoulder_Jerk_Mean', 'Left Elbow_Velocity_Mean',
       'Left Elbow_Acceleration_Mean', 'Left Elbow_Jerk_Mean',
       'Right Elbow_Velocity_Mean', 'Right Elbow_Acceleration_Mean',
       'Right Elbow_Jerk_Mean', 'Left Wrist_Velocity_Mean',
       'Left Wrist_Acceleration_Mean', 'Left Wrist_Jerk_Mean',
       'Right Wrist_Velocity_Mean', 'Right Wrist_Acceleration_Mean',
       'Right Wrist_Jerk_Mean', 'Left Hip_Velocity_Mean',
       'Left Hip_Acceleration_Mean', 'Left Hip_Jerk_Mean',
       'Right Hip_Velocity_Mean', 'Right Hip_Acceleration_Mean',
       'Right Hip_Jerk_Mean', 'Left Knee_Velocity_Mean',
       'Left Knee_Acceleration_Mean', 'Left Knee_Jerk_Mean',
       'Right Knee_Velocity_Mean', 'Right Knee_Acceleration_Mean',
       'Right Knee_Jerk_Mean', 'Left Ankle_Velocity_Mean',
       'Left Ankle_Acceleration_Mean', 'Left Ankle_Jerk_Mean',
       'Right Ankle_Velocity_Mean', 'Right Ankle_Acceleration_Mean',
       'Right Ankle_Jerk_Mean'
]

df = df[gait_features + ["Label"]].copy()

In [ ]:
# Convert labels to binary (0 = Healthy, 1 = MCI)
df["Label"] = df["Label"].map({"Healthy": 0, "MCI": 1})

# Step 1: Quantum Feature Extraction

In [ ]:
# Uninstall everything Qiskit-related to avoid version conflicts
!pip uninstall -y qiskit qiskit-terra qiskit-aer qiskit-ibm-provider

# Install Qiskit and IBM Provider versions that are compatible
!pip install qiskit==0.45.3 qiskit-ibm-provider==0.7.0

Found existing installation: qiskit 2.0.0
Uninstalling qiskit-2.0.0:
  Successfully uninstalled qiskit-2.0.0
Found existing installation: qiskit-terra 0.45.3
Uninstalling qiskit-terra-0.45.3:
  Successfully uninstalled qiskit-terra-0.45.3
Found existing installation: qiskit-aer 0.17.0
Uninstalling qiskit-aer-0.17.0:
  Successfully uninstalled qiskit-aer-0.17.0
Found existing installation: qiskit-ibm-provider 0.11.0
Uninstalling qiskit-ibm-provider-0.11.0:
  Successfully uninstalled qiskit-ibm-provider-0.11.0
  Using cached qiskit-0.45.3-py3-none-any.whl.metadata (12 kB)
  Using cached qiskit_terra-0.45.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached qiskit-0.45.3-py3-none-any.whl (9.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.9/242.9 kB 12.2 MB/s eta 0:00:00
Using cached qiskit_terra-0.45.3-cp38-abi3-manylinux_2_17_x86_64.manylinux20

In [ ]:
from google.colab import userdata
QIBM_API_KEY = userdata.get('QIBM_API_KEY')

In [ ]:
from qiskit_ibm_provider import IBMProvider

IBMProvider.save_account(QIBM_API_KEY, overwrite=True)
provider = IBMProvider()

In [ ]:
from qiskit_ibm_provider import least_busy

backend = least_busy(
    provider.backends(
        simulator=False,
        min_num_qubits=4,
        operational=True,
        dynamic_reprate_enabled=True
    )
)

print(f"Running on backend: {backend.name}")

Running on backend: ibm_kyiv


In [ ]:
!pip install pylatexenc

In [ ]:
from qiskit.primitives import Sampler
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
import matplotlib.pyplot as plt

# Number of qubits
num_qubits = 5

# Create a QuantumCircuit
qc = QuantumCircuit(num_qubits, num_qubits)

# Add Ry gates with symbolic parameters
for i in range(num_qubits):
    theta = Parameter(f'θ_{i}')
    qc.ry(theta, i)

# Add CNOT gates to form a ring entanglement
for i in range(num_qubits - 1):
    qc.cx(i, i + 1)
qc.cx(num_qubits - 1, 0)

# Add measurement operations
qc.measure(range(num_qubits), range(num_qubits))

In [ ]:
def quantum_feature_vector(features, backend_name="ibm_kyiv", num_qubits=5):
    from qiskit_ibm_provider import IBMProvider
    from qiskit.primitives import Sampler
    from qiskit import QuantumCircuit
    import numpy as np

    provider = IBMProvider()
    qc = QuantumCircuit(num_qubits, num_qubits)

    for i in range(num_qubits):
        qc.ry(features[i], i)

    for i in range(num_qubits - 1):
        qc.cx(i, i + 1)
    qc.cx(num_qubits - 1, 0)

    qc.measure(range(num_qubits), range(num_qubits))

    sampler = Sampler()
    job = sampler.run([qc])
    result = job.result()

    vec = [float(result.quasi_dists[0].get(i, 0.0)) for i in range(2 ** num_qubits)]
    return vec

In [ ]:
# --- CONFIG ---
CACHE_FILE = "quantum_cache.json"
num_qubits = 5

# --- Hashing utility ---
def hash_input(arr):
    return hashlib.md5(np.round(arr, 6).tobytes()).hexdigest()

# --- Load existing cache ---
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, "r") as f:
        cache = json.load(f)
else:
    cache = {}

# --- Safe quantum feature wrapper ---
def cached_quantum_feature_vector(features):
    key = hash_input(features)
    if key in cache:
        return cache[key]

    try:
        vec = quantum_feature_vector(features, num_qubits=num_qubits)
        cache[key] = vec

        # Save after each success
        with open(CACHE_FILE, "w") as f:
            json.dump(cache, f)

        print(f"✅ Processed: {key}")
        return vec

    except Exception as e:
        print(f"❌ Error on input {key}: {e}")
        return [0.0] * (2 ** num_qubits)  # placeholder if fail

# Step 2: Apply Quantum Encoding to Entire Dataset

In [ ]:
def extract_quantum_features_deduplicated(
    df,
    gait_features,
    quantum_feature_fn,
    num_qubits=5,
    cache_file="quantum_cache.json",
    save_every=10,
    output_csv="quantum_features_partial.csv"
):
    import numpy as np
    import pandas as pd
    import os
    import json
    import hashlib
    from tqdm import tqdm

    # Step 1: Round + deduplicate
    X_classical = df[gait_features].values
    X_classical_rounded = np.round(X_classical, 6)
    X_unique, idx_unique, idx_inverse = np.unique(
        X_classical_rounded, axis=0, return_index=True, return_inverse=True
    )

    print(f"🔍 Unique samples: {len(X_unique)} / {len(X_classical)} total")

    # Step 2: Load or initialize cache
    if os.path.exists(cache_file):
        with open(cache_file, "r") as f:
            cache = json.load(f)
    else:
        cache = {}

    # Step 3: Hashing function
    def hash_input(arr):
        return hashlib.md5(np.round(arr, 6).tobytes()).hexdigest()

    # Step 4: Safe call with cache
    def cached_quantum_call(features):
        key = hash_input(features)
        if key in cache:
            return cache[key]
        try:
            vec = quantum_feature_fn(features)
            cache[key] = vec
            with open(cache_file, "w") as f:
                json.dump(cache, f)
            return vec
        except Exception as e:
            print(f"❌ Error processing {key}: {e}")
            return [0.0] * (2 ** num_qubits)

    # Step 5: Extract features with incremental saving
    X_quantum_unique = []
    for i, x in enumerate(tqdm(X_unique)):
        X_quantum_unique.append(cached_quantum_call(x))

        # Save progress every N samples
        if (i + 1) % save_every == 0 or (i + 1) == len(X_unique):
            print(f"💾 Saving progress at sample {i + 1}")
            # Get mask of which unique rows were processed
            processed_mask = np.isin(idx_inverse, range(i + 1))

            # Reconstruct those rows
            partial_full = np.array(X_quantum_unique)[idx_inverse[processed_mask]]

            partial_df = pd.DataFrame(
                partial_full,
                columns=[f"Quantum_Feature_{j}" for j in range(2 ** num_qubits)]
            )
            partial_df["Label"] = df["Label"].values[processed_mask]

            # Save it
            partial_df.to_csv(output_csv, index=False)


    # Step 6: Reconstruct full dataset
    X_quantum_full = np.array(X_quantum_unique)[idx_inverse]

    # Step 7: Final DataFrame
    df_quantum = pd.DataFrame(
        X_quantum_full, columns=[f"Quantum_Feature_{i}" for i in range(2 ** num_qubits)]
    )
    df_quantum["Label"] = df["Label"].values

    # Final save
    df_quantum.to_csv(output_csv, index=False)
    print(f"✅ Final full CSV saved: {output_csv}")

    return df_quantum

In [ ]:
df_quantum = extract_quantum_features_deduplicated(
    df=df,
    gait_features=gait_features,
    quantum_feature_fn=quantum_feature_vector,
    num_qubits=5,
    cache_file="quantum_cache.json",
    save_every=10,
    output_csv="quantum_features_partial.csv"
)


🔍 Unique samples: 7188 / 137770 total





  0%|          | 0/7188 [00:00<?, ?it/s]

💾 Saving progress at sample 10


<ipython-input-31-78eac3ed4845>:19: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()



  0%|          | 11/7188 [00:03<39:44,  3.01it/s]


  0%|          | 12/7188 [00:06<1:19:48,  1.50it/s]


  0%|          | 13/7188 [00:10<2:01:50,  1.02s/it]


  0%|          | 14/7188 [00:13<2:48:02,  1.41s/it]


  0%|          | 15/7188 [00:16<3:25:51,  1.72s/it]


  0%|          | 16/7188 [00:19<3:58:46,  2.00s/it]


  0%|          | 17/7188 [00:22<4:25:18,  2.22s/it]


  0%|          | 18/7188 [00:25<4:52:46,  2.45s/it]


  0%|          | 19/7188 [00:28<5:08:18,  2.58s/it]


  0%|          | 20/7188 [00:31<5:17:38,  2.66s/it]

💾 Saving progress at sample 20


...

💾 Saving progress at sample 7180





100%|█████████▉| 7180/7188 [6:52:46<00:46,  5.82s/it]<ipython-input-31-78eac3ed4845>:19: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()



100%|█████████▉| 7181/7188 [6:52:49<00:35,  5.04s/it]


100%|█████████▉| 7182/7188 [6:52:52<00:26,  4.48s/it]


100%|█████████▉| 7183/7188 [6:52:55<00:20,  4.13s/it]


100%|█████████▉| 7184/7188 [6:52:59<00:15,  3.92s/it]


100%|█████████▉| 7185/7188 [6:53:02<00:11,  3.77s/it]


100%|█████████▉| 7186/7188 [6:53:06<00:07,  3.60s/it]


100%|█████████▉| 7187/7188 [6:53:09<00:03,  3.51s/it]

💾 Saving progress at sample 7188





100%|██████████| 7188/7188 [6:53:20<00:00,  3.45s/it]


✅ Final full CSV saved: /content/drive/MyDrive/QML/Features/pose/quantum_features_partial.csv


In [ ]:
# Save to CSV so we don’t lose data after restart
df_quantum.to_csv("mci_features_quantum.csv", index=False)

print("✅ Quantum features extracted and saved.")

✅ Quantum features extracted and saved.
